In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from mplsoccer.pitch import Pitch
from mplsoccer import Sbopen
import matplotlib as mpl
import json
import platform

In [2]:
# for referencing
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))

Python: 3.11.5
NumPy: 1.26.2
pandas: 2.1.4
matplotlib: 3.8.0


In [3]:
# Load configuration from config.json
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)

# database path
db_path = config.get('statsbomb_db_path')

# create an sqlalchemy engine to connect to the mysql database
mysql_hostname = config["mysql"]["connection"]["hostname"]
mysql_username = config["mysql"]["connection"]["username"]
mysql_password = config["mysql"]["connection"]["password"]
mysql_database = "statsbomb"
mysql_engine = create_engine(f'mysql+mysqlconnector://{mysql_username}:{mysql_password}@{mysql_hostname}/{mysql_database}')
import mysql.connector
def mysql_connect():
    return mysql.connector.connect(
        host= mysql_hostname,  
        user= mysql_username,  
        password= mysql_password, 
        database= mysql_database
    )

parser = Sbopen()
conn= mysql_connect()

In [5]:
comp_old = pd.read_sql_query(
    '''SELECT * 
    FROM competition
    ;
    ''', conn)
comp_old = comp_old.loc[comp_old.competition_gender == "male"]

In [49]:
comp_old.head()#[comp_old.country_name == 'International']

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,0,0,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,0,0,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,0,0,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,0,0,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,0,0,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00


In [7]:
# retrieve competitions
comp_new = parser.competition()
comp_new = comp_new.loc[comp_new.competition_gender == "male"]

comp_new

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-12-12T07:43:33.436182,None,None,2023-12-12T07:43:33.436182
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
62,2,44,England,Premier League,male,False,False,2003/2004,2023-11-14T11:20:59.843499,2021-06-13T16:17:31.694,None,2023-11-14T11:20:59.843499
63,12,27,Italy,Serie A,male,False,False,2015/2016,2023-12-13T17:32:46.423081,None,None,2023-12-13T17:32:46.423081
64,12,86,Italy,Serie A,male,False,False,1986/1987,2023-06-18T01:55:53.343752,None,None,2023-06-18T01:55:53.343752
65,55,43,Europe,UEFA Euro,male,False,True,2020,2023-02-24T21:26:47.128979,2023-04-27T22:38:34.970148,2023-04-27T22:38:34.970148,2023-02-24T21:26:47.128979


In [8]:
suff = ('_new','_old')
merged = comp_new.merge(comp_old, how='outer', on= ['competition_id','season_id'], suffixes=suff)
indeggs = merged.iloc[:,:2]

oldcol = merged.columns[merged.columns.str.endswith('_old')]
comp_old_merged = pd.concat([indeggs, merged[oldcol]], axis=1)
comp_old_merged.columns = [col.rsplit('_', 1)[0] for col in comp_old_merged.columns]

newcol = merged.columns[merged.columns.str.endswith('_new')]
comp_new_merged = pd.concat([indeggs, merged[newcol]], axis=1)
comp_new_merged.columns = [col.rsplit('_', 1)[0] for col in comp_new_merged.columns]

In [9]:
# detect changes 
comparison = comp_old_merged.compare(comp_new_merged, keep_shape=False, align_axis=1, result_names=('old', 'new')).iloc[:,len(indeggs.columns)*2:]

comparison_idx = pd.concat([indeggs, comparison], axis=1)

drop = [('competition_gender', 'old') , ('competition_gender', 'new'),('competition_youth', 'old'), ('competition_youth', 'new'),('competition_international', 'old'),('competition_international', 'new'),('season_name', 'old'),('season_name', 'new')]
nondrop = comparison_idx.columns[~comparison_idx.columns.isin((drop+indeggs.columns.tolist()))].tolist()
comparison_idx.drop(drop, axis=1,inplace=True)
comparison_idx.dropna(axis=0, how='all',subset=nondrop,inplace=True)
#comparison_idx.to_excel("comp.xlsx")
comparison_idx

,competition_id,season_id,"(match_updated, old)","(match_updated, new)","(match_updated_360, old)","(match_updated_360, new)","(match_available_360, old)","(match_available_360, new)","(match_available, old)","(match_available, new)"
0,9,27,2023-08-17T23:51:11.837478,2023-12-12T07:43:33.436182,NaN,NaN,NaN,NaN,2023-08-17T23:51:11.837478,2023-12-12T07:43:33.436182
18,16,276,2023-07-06T12:50:01.961048,2023-11-18T14:44:32.134540,NaN,NaN,NaN,NaN,2023-07-06T12:50:01.961048,2023-11-18T14:44:32.134540
20,87,268,2023-06-11T02:51:59.941964,2023-11-19T13:43:47.319278,NaN,NaN,NaN,NaN,2023-06-11T02:51:59.941964,2023-11-19T13:43:47.319278
23,43,106,2023-08-12T16:44:27.619465,2023-11-05T04:23:26.649917,2023-08-17T15:55:15.164685,2023-11-21T15:37:11.589616,2023-08-17T15:55:15.164685,2023-11-21T15:37:11.589616,2023-08-12T16:44:27.619465,2023-11-05T04:23:26.649917
32,11,90,2023-07-26T14:11:01.312143,2023-11-15T10:25:53.245065,2023-07-26T14:15:15.217027,2023-11-15T10:29:14.475865,2023-07-26T14:15:15.217027,2023-11-15T10:29:14.475865,2023-07-26T14:11:01.312143,2023-11-15T10:25:53.245065
34,11,4,2023-08-03T02:24:43.761907,2023-11-05T09:50:49.746684,NaN,NaN,NaN,NaN,2023-08-03T02:24:43.761907,2023-11-05T09:50:49.746684
36,11,2,2023-07-25T00:14:55.260536,2023-11-14T18:26:44.671413,NaN,NaN,NaN,NaN,2023-07-25T00:14:55.260536,2023-11-14T18:26:44.671413
37,11,27,2023-07-24T21:52:42.890908,2023-09-20T17:21:32.111535,NaN,NaN,NaN,NaN,2023-07-24T21:52:42.890908,2023-09-20T17:21:32.111535
38,11,26,2023-07-24T21:50:15.215072,2023-09-20T17:22:28.925900,NaN,NaN,NaN,NaN,2023-07-24T21:50:15.215072,2023-09-20T17:22:28.925900
43,11,21,2023-04-19T10:48:01.770383,2023-11-29T10:55:10.039117,NaN,NaN,NaN,NaN,2023-04-19T10:48:01.770383,2023-11-29T10:55:10.039117


In [14]:
matches = pd.DataFrame()
for i, comp in comparison_idx.iterrows():
    print("Entered:",i)
    if i == 0:
        matches = parser.match(competition_id=comp.competition_id, season_id=comp.season_id)
    else:
        stg = parser.match(competition_id=comp.competition_id, season_id=comp.season_id)
        matches = pd.concat([matches, stg], axis=0, ignore_index=True)

match_list = matches.match_id.unique()

Entered: 0
Entered: 18
Entered: 20
Entered: 23
Entered: 32
Entered: 34
Entered: 36
Entered: 37
Entered: 38
Entered: 43
Entered: 48
Entered: 51
Entered: 52
Entered: 53
Entered: 54
Entered: 55
Entered: 57
Entered: 58
Entered: 59


In [15]:
matches_old = pd.read_sql_query(
    '''SELECT *
    FROM `match`;
    ''', conn)

In [44]:
changed_matches = matches_old[matches_old.match_id.isin(matches.match_id)] 
changed_matches.competition_name.value_counts()

competition_name
La Liga             563
Premier League      418
Serie A             380
1. Bundesliga       306
FIFA World Cup       64
Champions League      1
Copa del Rey          1
Liga Profesional      1
Name: count, dtype: int64

In [47]:
new_matches = matches[~matches.match_id.isin(matches_old.match_id)]
new_matches.competition_name.value_counts()

competition_name
Ligue 1                58
Major League Soccer     6
Name: count, dtype: int64

In [46]:
matches.to_sql('match', mysql_engine, if_exists='append', index=False) # insert all new matches.
cursor = conn.cursor()
delete_query = """
DELETE FROM `match`
WHERE match_id IN (
    SELECT match_id
    FROM (
        SELECT match_id,
                ROW_NUMBER() OVER (PARTITION BY match_id ORDER BY last_updated DESC) as rn
        FROM `match`
    ) tmp
    WHERE rn > 1
)
"""
cursor.execute(delete_query)
conn.commit()

print(cursor.rowcount, "rows deleted.")
cursor.close()
matches.to_sql('match', mysql_engine, if_exists='append', index=False)

5330 rows deleted.


1798

In [48]:
# verify there are no duplicate match_ids
pd.read_sql_query("""SELECT match_id
FROM (
    SELECT match_id,
            ROW_NUMBER() OVER (PARTITION BY match_id ORDER BY last_updated DESC) as rn
    FROM `match`
) tmp
WHERE rn > 1""",conn)


,match_id
